In [10]:
import requests
base_url = 'https://m.aliexpress.com/ajaxapi/EvaluationSearchAjax.do?type=all&pageSize=80&country=US'
def importReviews(doc_complete, product_id):
    totalPage = 10000000000000000000
    n = 1
    while (n<=totalPage):
        #print(n)
        url = base_url + "&productId="+str(product_id)+"&index="+str(n)
        #print(url)
        resp = requests.get(url)
        if resp.status_code != 200:
            # This means something went wrong.
            raise ApiError('GET /EvaluationSearchAjax/ {}'.format(resp.status_code))
        resp_json = resp.json()
        doc_complete_n = [review["buyerTranslationFeedback"] 
                          for review in resp_json["evaViewList"] 
                          if "buyerTranslationFeedback" in review.keys() 
                          #and float(review["buyerEval"])<50
                         ]
        doc_complete= doc_complete + doc_complete_n
        if (n==1):
            #print(resp_json)
            totalPage = int(resp_json["totalPage"])
            print(str(product_id)+" totalPage "+str(totalPage))
        print(".", end='')    
        n=n+1
    return doc_complete

In [11]:
#Xiaomi Redmi 4 Pro Prime
#  32752699189 Doogee x5 max 
# 32757712697 lenovo a5600 
# 32798494733 Lenovo ZUK Z2 
# 32791453530 ONEPLUS 3 T A3003 
# 32671947390  Blackview BV6000 
# 32719969124 Xiaomi Mi Max Mimax Pro 
# 32799364813 Oukitel U20 Plus Android 6.0
# 32611785137 Xiaomi Mi5 M5 Mi 5 Prime
# 32763320059
product_ids = [
32796300206,
32797368004,
32660593076,
32763320059,
32752699189,
32757712697,
32798494733,
32791453530,
32671947390,
32719969124,
32799364813,
32611785137,
32763320059]
doc_complete = []
for product_id in product_ids:
    doc_complete=importReviews(doc_complete,product_id)
print("count of reviews: "+ str(len(doc_complete)))



32796300206 totalPage 76
............................................................................32797368004 totalPage 6
......32660593076 totalPage 12
............32763320059 totalPage 123
...........................................................................................................................32752699189 totalPage 11
...........32757712697 totalPage 7
.......32798494733 totalPage 10
..........32791453530 totalPage 7
.......32671947390 totalPage 8
........32719969124 totalPage 8
........32799364813 totalPage 6
......32611785137 totalPage 50
..................................................32763320059 totalPage 123
...........................................................................................................................count of reviews: 100


In [12]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    #stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    #punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    tokens = word_tokenize(doc.lower())
    normalized = " ".join(lemma.lemmatize(token) for token in tokens if token not in stop and token not in exclude)
    return normalized

doc_clean = [clean(doc) for doc in doc_complete]   
print(doc_clean)

['phone came quickly within two week use within 4 day positive emotion cause time came third charging phone stopped charging change usb cable charging pc reset factory help dialogue seller lasted 2 week description problem sending video end offered try change usb cable essentially problem similar solution problem new phone offered given diagnosis turned loose contact battery charging board connecting back everything worked phone began charge recharged stop know fix problem real thanks diagnosis especially thank reliable new phone manufacturer `` buying get ready adventure ... ...', 'ordered hot glass came film seller respond', 'know begin ... first time history shopping ali came marriage case marriage stitch top dead mm 15 rest responded old nokia 5300. unsubscribed sell refused change tipo repair fight week without removing sort different video psihanul opened debate return money condition took last hour dispute solution offered 0 repair sent body samara region got trusted store perso

In [13]:
import nltk

full_text = " ".join([review for review in doc_clean])

tokens = nltk.word_tokenize(full_text)

tagged = nltk.pos_tag(tokens)
print(tagged)

[('phone', 'NN'), ('came', 'VBD'), ('quickly', 'RB'), ('within', 'IN'), ('two', 'CD'), ('week', 'NN'), ('use', 'NN'), ('within', 'IN'), ('4', 'CD'), ('day', 'NN'), ('positive', 'JJ'), ('emotion', 'NN'), ('cause', 'NN'), ('time', 'NN'), ('came', 'VBD'), ('third', 'JJ'), ('charging', 'VBG'), ('phone', 'NN'), ('stopped', 'VBD'), ('charging', 'VBG'), ('change', 'NN'), ('usb', 'JJ'), ('cable', 'NN'), ('charging', 'VBG'), ('pc', 'NN'), ('reset', 'NN'), ('factory', 'NN'), ('help', 'NN'), ('dialogue', 'VB'), ('seller', 'NN'), ('lasted', 'VBD'), ('2', 'CD'), ('week', 'NN'), ('description', 'NN'), ('problem', 'NN'), ('sending', 'VBG'), ('video', 'JJ'), ('end', 'NN'), ('offered', 'VBD'), ('try', 'NNS'), ('change', 'VBP'), ('usb', 'JJ'), ('cable', 'NN'), ('essentially', 'RB'), ('problem', 'NN'), ('similar', 'JJ'), ('solution', 'NN'), ('problem', 'NN'), ('new', 'JJ'), ('phone', 'NN'), ('offered', 'VBD'), ('given', 'VBN'), ('diagnosis', 'NN'), ('turned', 'VBD'), ('loose', 'JJ'), ('contact', 'JJ'), (

In [14]:
#nouns = [token for token, pos in tagged if pos.startswith('N')]
#print(nouns)
  	
grammar = r"""
  NOUN_PHRASE:  {<CD|IN|DT|JJ|PP|RB|\$>*<N.*>+<V.*>*<CD|IN|DT|JJ|PP|RB|\$>*<N.*>*}  
"""
cp = nltk.RegexpParser(grammar)
noun_phrase_tagged = cp.parse(tagged)
print(noun_phrase_tagged)
#doc_complete = nouns

(S
  (NOUN_PHRASE
    phone/NN
    came/VBD
    quickly/RB
    within/IN
    two/CD
    week/NN
    use/NN)
  (NOUN_PHRASE
    within/IN
    4/CD
    day/NN
    positive/JJ
    emotion/NN
    cause/NN
    time/NN)
  came/VBD
  third/JJ
  charging/VBG
  (NOUN_PHRASE phone/NN stopped/VBD charging/VBG change/NN)
  (NOUN_PHRASE
    usb/JJ
    cable/NN
    charging/VBG
    pc/NN
    reset/NN
    factory/NN
    help/NN)
  dialogue/VB
  (NOUN_PHRASE
    seller/NN
    lasted/VBD
    2/CD
    week/NN
    description/NN
    problem/NN)
  sending/VBG
  (NOUN_PHRASE video/JJ end/NN offered/VBD try/NNS)
  change/VBP
  (NOUN_PHRASE usb/JJ cable/NN essentially/RB problem/NN)
  (NOUN_PHRASE similar/JJ solution/NN problem/NN new/JJ phone/NN)
  offered/VBD
  given/VBN
  (NOUN_PHRASE
    diagnosis/NN
    turned/VBD
    loose/JJ
    contact/JJ
    battery/NN)
  charging/VBG
  (NOUN_PHRASE board/NN connecting/VBG)
  back/RP
  (NOUN_PHRASE everything/NN worked/VBD phone/NN)
  began/VBD
  (NOUN_PHRASE charge

In [15]:
def traverse(t,noun_phrases):
    try:
        t.label()
    except AttributeError:
        n=0
    else:
        if t.label() == 'NOUN_PHRASE': 
            [noun_phrases.append(word) for (word,pos) in t.leaves() if pos.startswith("N")]
            #print(statement)
            #noun_phrases = noun_phrases.append(noun_phrase)
        # Now we know that t.node is defined
        #print('(', t.label(), end=" ")
        for child in t:
            traverse(child,noun_phrases)
        #print(')', end=" ")
    return noun_phrases

noun_phrases=[]
nouns=traverse(noun_phrase_tagged,noun_phrases)
print(nouns)
doc_complete = nouns

['phone', 'week', 'use', 'day', 'emotion', 'cause', 'time', 'phone', 'change', 'cable', 'pc', 'reset', 'factory', 'help', 'seller', 'week', 'description', 'problem', 'end', 'try', 'cable', 'problem', 'solution', 'problem', 'phone', 'diagnosis', 'battery', 'board', 'everything', 'phone', 'charge', 'stop', 'fix', 'problem', 'thanks', 'phone', 'manufacturer', 'adventure', 'glass', 'film', 'seller', 'respond', 'begin', 'time', 'history', 'shopping', 'ali', 'marriage', 'case', 'marriage', 'stitch', 'top', 'mm', 'rest', 'sell', 'change', 'tipo', 'repair', 'week', 'sort', 'video', 'psihanul', 'return', 'money', 'condition', 'hour', 'dispute', 'solution', 'repair', 'body', 'region', 'person', 'receipt', 'intend', 'phone', 'sign', 'use', 'etc', 'fact', 'send', 'factory', 'xiaomi', 'warranty', 'repair', 'time', 'evidence', 'photo', 'video', 'condition', 'body', 'customer', 'service', 'evidence', 'protection', 'buyer', 'petition', 'phone', 'class', 'work', 'problem', 'sensor', 'work', 'seller', '

In [16]:
from nltk import FreqDist
import nltk
#print(doc_clean)
#words = nltk.tokenize.word_tokenize(doc_clean)
all_reviews_in_one_string = " ".join([doc for doc in doc_clean])

words = nltk.tokenize.word_tokenize(all_reviews_in_one_string)
fdist = FreqDist(noun_phrases) 
frequent_words_freq = fdist.most_common(100)
print(frequent_words_freq )
#fdist.plot()
frequent_words= [w for (w,c) in frequent_words_freq]
#print(frequent_words)



[('seller', 136), ('phone', 133), ('time', 41), ('day', 40), ('money', 31), ('order', 31), ('dispute', 28), ('week', 27), ('month', 25), ('case', 19), ('problem', 17), ('work', 16), ('battery', 14), ('glass', 13), ('video', 13), ('parcel', 12), ('delivery', 12), ('color', 12), ('refund', 11), ('film', 10), ('send', 10), ('service', 10), ('track', 10), ('review', 10), ('repair', 9), ('quality', 9), ('payment', 9), ('everything', 8), ('ali', 8), ('photo', 8), ('screen', 8), ('model', 8), ('recommend', 8), ('end', 7), ('post', 7), ('box', 7), ('star', 7), ('display', 7), ('packaging', 7), ('use', 6), ('body', 6), ('package', 6), ('come', 6), ('anything', 6), ('result', 6), ('advise', 6), ('purchase', 6), ('mail', 6), ('pay', 6), ('request', 6), ('silicone', 6), ('number', 6), ('solution', 5), ('charge', 5), ('shipment', 5), ('get', 5), ('contact', 5), ('course', 5), ('aliekspress', 5), ('shipping', 5), ('way', 5), ('firmware', 5), ('question', 5), ('complaint', 5), ('message', 5), ('story

def clean(doc):
    tokens = word_tokenize(doc)
    only_frqt_words = " ".join([token for token in tokens if token in frequent_words])
    return only_frqt_words 

#doc_clean2 = [clean(doc) for doc in doc_clean] 
doc_clean2 = [clean(doc).split() for doc in doc_clean] 
#print(doc_clean2)

Now find snippets in the review with these frequent nouns

In [17]:
def traverse(t,noun,phrases):
    try:
        t.label()
    except AttributeError:
        n=0
    else:
        if t.label() == 'NOUN_PHRASE': 
            phrase = " ".join([word for (word,pos) in t.leaves()])
            if noun in phrase.split():
                phrases.append(phrase)
                #print(phrase)
        for child in t:
            traverse(child,noun,phrases)
    return phrases

from nltk.sentiment.vader import SentimentIntensityAnalyzer

dict_noun_abs_sentiment = {}
for noun in frequent_words:
    phrases=[]
    phrases=traverse(noun_phrase_tagged,noun,phrases)
    print(phrases[1:3])
    #doc_complete = nouns

    sid =  SentimentIntensityAnalyzer()
    sentiments = sum(abs(sid.polarity_scores(phrase)["compound"]) for phrase in phrases)/len(phrases)
    print(noun + " has absolute avg sentiment " + str(sentiments))
    dict_noun_abs_sentiment[noun]=sentiments

['hot glass came film seller respond', 'properly seller want recognize something']
seller has absolute avg sentiment 0.2238403100775194
['phone stopped charging change', 'similar solution problem new phone']
phone has absolute avg sentiment 0.14661904761904762
['first time history shopping ali came marriage case marriage stitch top', 'warranty repair накрыло second time']
time has absolute avg sentiment 0.19598205128205132
['speedily even delivery russia 5 day case', 'back phone came 25 day']
day has absolute avg sentiment 0.10429487179487179
['dispute aliekspress taken away seller end phone money', 'empty thanks aliekspressu money']
money has absolute avg sentiment 0.16898387096774192
['problem seller sent long order', 'pretty recommend seller mess order seller course super terrible seller contact']
order has absolute avg sentiment 0.3139964285714286
['dispute aliekspress taken away seller end phone money', 'dispute maximum agree 10']
dispute has absolute avg sentiment 0.4399499999999

firmware has absolute avg sentiment 0.16076
['question buying phone', 'question purchase good']
question has absolute avg sentiment 0.08808
['battery case unique seller complaint standard', 'breakfast постил pattern addressed phone complaint']
complaint has absolute avg sentiment 0.296
['week seller completely игнорирывал message', 'disappointed unresponsive even bad seller message']
message has absolute avg sentiment 0.153
['video emitatsii packaging story youtube 4pda seller кантактировал', 'true story result waiting parcel']
story has absolute avg sentiment 0.0843
['respond seller offered send back expense', 'respond message wrote one day']
respond has absolute avg sentiment 0.0
['one phone come marriage display', 'one phone come marriage display']
marriage has absolute avg sentiment 0.06743333333333333
['unpacking parcel seller refused pay return shipping', 'back expense return phone seller service aliexpress accept solution refund']
return has absolute avg sentiment 0.193516666666

In [18]:
print( [ (key,'{:01.2f}'.format(val)) for (key,val) in dict_noun_abs_sentiment.items() if val >0.1])

[('seller', '0.22'), ('phone', '0.15'), ('time', '0.20'), ('day', '0.10'), ('money', '0.17'), ('order', '0.31'), ('dispute', '0.44'), ('week', '0.16'), ('month', '0.18'), ('case', '0.13'), ('problem', '0.36'), ('work', '0.23'), ('battery', '0.25'), ('parcel', '0.14'), ('delivery', '0.28'), ('color', '0.31'), ('refund', '0.33'), ('film', '0.12'), ('send', '0.17'), ('service', '0.23'), ('track', '0.16'), ('review', '0.30'), ('quality', '0.18'), ('payment', '0.13'), ('everything', '0.18'), ('ali', '0.25'), ('photo', '0.17'), ('screen', '0.20'), ('recommend', '0.39'), ('end', '0.13'), ('post', '0.14'), ('box', '0.12'), ('display', '0.32'), ('packaging', '0.12'), ('use', '0.18'), ('body', '0.22'), ('come', '0.22'), ('anything', '0.25'), ('result', '0.30'), ('advise', '0.21'), ('mail', '0.12'), ('pay', '0.23'), ('request', '0.19'), ('number', '0.13'), ('solution', '0.29'), ('shipment', '0.19'), ('contact', '0.29'), ('course', '0.20'), ('aliekspress', '0.32'), ('shipping', '0.24'), ('way', '0

# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean2)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean2]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=50, id2word = dictionary, passes=10)

print(ldamodel.print_topics(num_topics=50, num_words=5))

model = gensim.models.Word2Vec(doc_clean)

print(model)

model.similarity('camera', 'battery')